In [ ]:
"""
NOTEBOOK 07: MODELADO DE DINÁMICAS MICROVASCULARES
===================================================
Construir modelos predictivos de la perfusión tisular a partir
de características espacio-temporales y estadísticas de flujo.

Técnicas:
- Análisis de series temporales (autocorrelación, espectral)
- Modelado de dependencia con volumen de plasma
- Validación cruzada con métricas clínicas

Entrada: Características espacio-temporales (notebook 06)
Salida: Modelos predictivos, coeficientes, métricas de bondad

Autor: Tesis Doctoral - Análisis Automatizado de Dinámicas Microvasculares
"""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from scipy import signal, stats
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression

# Configuración de visualización
sns.set_style("whitegrid")
sns.set_palette("Set2")
plt.rcParams['figure.figsize'] = (14, 8)
plt.rcParams['font.size'] = 11
plt.rcParams['axes.labelsize'] = 12
plt.rcParams['axes.titlesize'] = 13
plt.rcParams['legend.fontsize'] = 10

In [ ]:
# Save dynamics models
dynamics_df = pd.DataFrame(dynamics_models).T
output_path = Path('/Users/luisestebanbaldasseroni/LuisEsteban/tesis/microcirculation-analysis/src/data/dynamics_models.csv')
dynamics_df.to_csv(output_path)

print(f"\nDynamics models saved to: {output_path}")
print("\nReady for clinical metrics analysis in Notebook 08")

In [ ]:
# Visualize temporal trends
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
fig.suptitle('Microvascular Dynamics Over Time', fontsize=14, fontweight='bold')

# Patient selection for detailed plots
patients_with_trend = [p for p in dynamics_models.keys() if dynamics_models[p]['num_measurements'] >= 3]

ax = axes[0, 0]
for patient in patients_with_trend[:3]:  # Show first 3
    patient_data = df_full[df_full['patient'] == patient].sort_values('day_num')
    ax.plot(patient_data['day_num'], patient_data['svd'], marker='o', label=patient, linewidth=2)
ax.set_xlabel('Day')
ax.set_ylabel('Small Vessel Density (SVD)')
ax.set_title('SVD Temporal Trend')
ax.legend()
ax.grid(True, alpha=0.3)

ax = axes[0, 1]
for patient in patients_with_trend[:3]:
    patient_data = df_full[df_full['patient'] == patient].sort_values('day_num')
    ax.plot(patient_data['day_num'], patient_data['tvd'], marker='s', label=patient, linewidth=2)
ax.set_xlabel('Day')
ax.set_ylabel('Total Vessel Density (TVD)')
ax.set_title('TVD Temporal Trend')
ax.legend()
ax.grid(True, alpha=0.3)

# Distribution of trend slopes
ax = axes[1, 0]
trends_svd = [dynamics_models[p]['svd_trend'] for p in patients_with_trend]
ax.bar(patients_with_trend, trends_svd, color=['green' if t > 0 else 'red' for t in trends_svd], alpha=0.7)
ax.axhline(y=0, color='k', linestyle='-', linewidth=0.5)
ax.set_ylabel('SVD Trend (per day)')
ax.set_title('SVD Temporal Trend Slopes')
ax.grid(True, alpha=0.3, axis='y')
plt.setp(ax.xaxis.get_majorticklabels(), rotation=45)

# Model quality summary
ax = axes[1, 1]
model_summary = pd.DataFrame([{'Patient': k, 'Trend': v['svd_trend'], 'Mean SVD': v['mean_svd']} 
                             for k, v in dynamics_models.items() if v['num_measurements'] >= 3])
if len(model_summary) > 0:
    ax.scatter(model_summary['Mean SVD'], model_summary['Trend'], s=150, alpha=0.6, color='steelblue')
    for idx, row in model_summary.iterrows():
        ax.annotate(row['Patient'], (row['Mean SVD'], row['Trend']), fontsize=8)
ax.set_xlabel('Mean SVD')
ax.set_ylabel('SVD Trend')
ax.set_title('Baseline vs Temporal Trend')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('/Users/luisestebanbaldasseroni/LuisEsteban/tesis/microcirculation-analysis/src/data/microvascular_dynamics.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# Load annotations with temporal data
annotations_path = Path('/Users/luisestebanbaldasseroni/LuisEsteban/tesis/microcirculation-analysis/src/data/annotations_processed.csv')
df_full = pd.read_csv(annotations_path)

# Load spatiotemporal features
spacetime_path = Path('/Users/luisestebanbaldasseroni/LuisEsteban/tesis/microcirculation-analysis/src/data/spatiotemporal_features.csv')
df_spacetime = pd.read_csv(spacetime_path)

print(f"Analyzing dynamics for {df_full['patient'].nunique()} patients")

# Build temporal models for each patient
dynamics_models = {}

for patient in df_full['patient'].unique():
    patient_data = df_full[df_full['patient'] == patient].sort_values('day_num')
    
    if len(patient_data) > 2:  # Need at least 3 measurements
        days = patient_data['day_num'].values
        svd = patient_data['svd'].values
        tvd = patient_data['tvd'].values
        total_vessels = patient_data['total_vessels'].values
        
        # Linear regression for temporal trends
        z_svd = np.polyfit(days, svd, 1)
        z_tvd = np.polyfit(days, tvd, 1)
        z_vessels = np.polyfit(days, total_vessels, 1)
        
        # Compute correlation coefficients
        r_svd, p_svd = stats.linregress(days, svd)[:2]
        
        dynamics_models[patient] = {
            'num_measurements': len(patient_data),
            'svd_trend': z_svd[0],  # slope
            'tvd_trend': z_tvd[0],
            'vessel_trend': z_vessels[0],
            'svd_pvalue': p_svd,
            'mean_svd': np.mean(svd),
            'mean_tvd': np.mean(tvd),
            'days_range': days[-1] - days[0]
        }

# Display dynamics
print("\nMicrovascular Dynamics Models:")
print("-" * 100)

for patient, model in dynamics_models.items():
    print(f"\n{patient}:")
    print(f"  Measurements: {model['num_measurements']}")
    print(f"  SVD trend (per day): {model['svd_trend']:+.3f} " + 
          f"({'INCREASING' if model['svd_trend'] > 0 else 'DECREASING'})")
    print(f"  TVD trend (per day): {model['tvd_trend']:+.3f}")
    print(f"  Vessel count trend: {model['vessel_trend']:+.2f} per day")

# Notebook 07: Microvascular Dynamics Modeling

Models temporal dynamics of microvascular perfusion and identifies patterns of hemodynamic behavior.

# 07 - Modelado Dinámico

## Objetivo
- Entrenar modelo secuencial (GRU, LSTM o Transformer)
- Predecir dinámicas microvasculares

## Dataset
Vectores de métricas por video (series temporales)


In [ ]:
from src.models.rnn import MicroDynamicsRNN

model = MicroDynamicsRNN(input_dim=3, hidden_dim=64)
loss_fn = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

for x_batch, y_batch in train_loader:
    y_pred = model(x_batch)
    loss = loss_fn(y_pred, y_batch)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
